# Scaling out (multi-nodes + multi-GPUs)

Lightning integrates, standardly, a lot of options to go for multi-nodes, multi-GPUs training on a cluster. For HPC-grade cluster, it would be fooling not to leverage Interconnect capabilities. Lightning makes it easy to launch with SLURM at negligeable cost.

### SLURM

SLURM is an Open Source, flexible job scheduler used to manage resources in an HPC context, in 3 keys functions:
* Exclusive or non-exclusive resource allocation system for a specific time period
* A tool for executing and monitoring jobs on a set of allocated resources
* A scheduling system that manages contention for resources

### Quick-guide to Resources Allocation and Job Submission

**This part is just a brief introduction to SLURM not intended as a comprehensive review.**

A few commands:

* Listing visible cluster resources with `sinfo <options>`
* Allocate resources with `salloc <resources_type_and_options>`
* Run commands directly on allocated resources with an `srun <command>`. This will end-up building the environment
* Run an ensemble of commands with a call to `sbatch <script>`

echo ${SLURM_NODEID} && cat /etc/docker/daemon.json